In [1]:
import pandas as pd
import numpy as np

# Load the data

In [2]:
file_list = pd.read_csv('Data/file_list.csv')

In [3]:
file_list.head()

,file_name,Case_PTC_No,death_date
0,2020-1519BR01.mzML,20201519,6/26/2020
1,2020-1520BG01.mzML,20201520,6/27/2020
2,2020-1521BG01.mzML,20201521,6/25/2020
3,2020-1522BG01.mzML,20201522,6/25/2020
4,2020-1523BG01.mzML,20201523,6/30/2020
